# LSTM Time-Series Forecasting for SPY 
**Project:** Time-series forecasting for equities using LSTM

**Author:** Khalid Hajeer

**Last updated:** September 13, 2025

## Table of Contents
1. [Introduction](#1-introduction)
2. [Setup & Reproducibility](#2-setup--reproducibility)  
3. [Data Collection & Preprocessing](#3-data-collection--preprocessing)  
4. [Exploratory Data Analysis (EDA)](#4-exploratory-data-analysis-eda)  
5. [Feature Engineering](#5-feature-engineering)
6. [Systematic Feature Selection](#6-systematic-feature-selection)  
7. [Baselines](#7-baselines)  
8. [LSTM Data Preparation & Architecture](#8-lstm-data-preparation--architecture)  
9. [Training & Validation](#9-training--validation) 
10. [Test Evaluation & Backtest](#10-test-evaluation--backtest)
11. [Conclusion](#11-conclusion)
12. [Appendex](#12-Appendex)


## 1. Introduction

The **SPDR S&P 500 ETF Trust (SPY)** tracks the performance of the S&P500 index - a benchmark of 500 leading publicly listed companies in the United States. SPY is highly liquid and often used as a proxy for the overall U.S. stock market.

> **Note:** Long run performance figures vary by window and methodology. In this project, SPY serves primarily as a liquid benchmark for modelling and backtesting rather than as a statement about historical returns.

Finance time-series forecasting is challenging: signals are weak, noisy, and non-stationary. Patterns can emerge, decay, or invert with regime shifts and market feedback. We therefore use strict chronological splits, early stopping on validation correlation, refitting before test, and cost-aware backtesting to assess whether any learned structure is robust.

### Objectives
Build and evaluate a **Long Short-Term Memory (LSTM)** network (a type of Recurrent Neural Network (RNN)) to predict the **next day return** of SPY. We model **log returns** instead of raw prices for better stationarity and comparability across time.
$$
r_t = \ln \left(\frac{C_t}{C_{t-1}}\right) = \ln C_t - \ln C_{t-1}.
$$
where:
- $r_t$ is the log return at day $t$
- $C_t$ is close at day $t$
- $C_{t-1}$ is close at day $t-1$

From a predicted log return $\hat{r}_{t+1}$, we recover the next day close:
$$
\hat{C}_{t+1} = C_t \cdot e^{\hat{r}_{t+1}}
$$

### Methodology Overview

**Setup & Reproducibility**  
    Import libraries; set deterministic seeds; define a central config (paths, splits, dates, costs).

**Data Collection & Preprocessing**  
    Pull daily OHLCV (Open, High, Low, Close, Volume) from Alpaca; validate monotonic timestamps; enforce dtypes; handle missing values; verify no lookahead in any merge; persist a clean, versioned dataset.

**Exploratory Data Analysis (EDA)**  
    Plot distributions and volatility; examine regime hints; inspect missingness and outliers.

**Feature Engineering**   
    Build lagged/rolling features (momentum, volatility, volume, statistical); shift all predictors to t-1 to avoid lookahead; persist a versioned feature catalog.

**Systematic Feature Selection**   
    Rank features on train only; freeze the selected set for modelling.

**Baselines**  
    Train Ridge and XGBoost baselines plus a naïve persistence/momentum rule; evaluate with identical splits and metrics (Pearson correlation, MSE, and trading PnL metrics).

**LSTM Data Preparation & Architecture**   
    Fit scalers on train only; create a sliding window sequence; specify LSTM layers, units, dropout with fixed seeds; checkpoint models; ensure target alignment.

**Training, Validation & Model Selection**  
    Train with early stop on validation Pearson correlation; record the best epoch along with its weights and metrics. The best epoch is then used to refit the model from scratch on combined train+val set (without early_stopping), after which test predictions are generated and all artefacts are archived.

**Backtest & Test Evaluation**  
    Map signal -> position; compare against baselines and a label-shuffle control; report Sharpe, MaxDD, Turnover; plot a $1k wealth chart for the test set.

**Diagnostics**  
    Inspect rolling correlation/Sharpe, error-by-volatility profiles, and residual ACF; review feature usage/coverage and data leakage checks.

**Conclusions**  
    Summarise findings; discuss limitations; suggest next steps.

### Why LSTM?
LSTMs capture patterns in sequential and temporal data by learning which parts of past information are most relevant for future predictions. They are a type of Recurring Neural Network (RNN) designed to overcome the vanishing gradient problem, where gradients become extremely small during backpropagation through time, making it difficult for standard RNNs to learn long-term dependencies.
By incorporating gating mechanisms, LSTMs can retain important information over longer sequences, making them particularly suited for stock market time-series forecasting.

### Scope & Assumptions
- The approach generalises to any Alpaca-supported ticker; performance will vary by asset, liquidity, and regime.
- Corporate actions (splits/dividends) and survivorship biases must be handled explicitly.
- Inputs are historical OHLCV + engineered features only (no macro/news)
- All transformations are fit on training data only and applied forward to avoid leakage.

### Expected Output
- A trained LSTM model that predicts next-day log returns.
- A test-only backtest report with after-cost metrics, $1k wealth chart, baselines, and a shuffle control.
- Visual comparisons: predicted vs. realised returns (and derived prices).
- One-step-ahead forecast for the next trading day
- A concise tearsheet and a longer Methods PDF

### Financial Disclaimer
This project is for **educational purposes only** and is **not financial advice**.
Model outputs are uncertain and should not be used for live trading without independent verification and risk controls.

> "If you were to give a monkey a typewriter and let him randomly hit keys forever, then he would eventually type Shakespeare's Hamlet - word for word."

## 2. Libraries

In [ ]:
# Core libraries 
import os, sys, random
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import json
from pathlib import Path

# Machine learning libraries
import tensorflow as tf
from tensorflow import keras
print(tf.config.list_physical_devices('GPU'))
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")
print(f"TensorFlow version: {tf.__version__}")

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, mean_squared_error

# Visualization libraries
import matplotlib.pyplot as plt

# Project modules
# Add parent directory (so Python can see src/)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
# TODO: Update requirements.txt at the end
from src.data import get_stock_data, align_calendar
from src.features import make_targets, build_feature_matrix
from src.validate import run_all_validations
from src.metrics import *
from src.backtest import single_asset_backtest
from src.models.baseline import *
from src.utils import *

# Set seed for reproducibility
GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)

pd.options.display.float_format = "{:.6f}".format
print("Imports ready!")

## 3. Data Collection & Preprocessing

First, let's fetch the data we'll be working with.

We'll use the Alpaca API credentials from the environment and fetch 10 years of daily OHLCV (+ VWAP, trade_count) data for SPY.

To save time on future runs, the dataet is cached locally at 'data/raw/SPY.parquet'.
If the cache is missing or we set 'FORCE_REFRESH=True', the data will be pulled fresh from Alpaca and saved to that path.

After fetching, we make the data is clean and ready:
- enforce a UTC 'DatetimeIndex' that is monotonic and unique,
- coerce numeric dtypes, and remove duplicates/missing values,

Finally, we align the dataset with the trading calendar, ensuring no gaps remain before downstream feature engineering and backtesting.

In [ ]:
# Load API credentials from .env file (ALPACA_API_KEY, ALPACA_SECRET_KEY)
load_dotenv()

symbol = "SPY"
CACHE_PATH = "../data/raw/SPY.parquet" # local cache location
FORCE_REFRESH = True # set to True to fetch fresh data

# Fetch data from Alpaca if needed, else load from cache
# Output is a Pandas DataFrame with UTC DatetimeIndex
df = get_stock_data(
    symbol=symbol,
    years=5,
    path=CACHE_PATH,
    force_refresh=FORCE_REFRESH,
    feed="iex"
)

# Align calendar: ensuring  sorted, unique UTC DatetimeIndex
df = align_calendar(df)

## 4. Exploratory Data Analysis (EDA)

Before feature engineering, let's explore the dataset and get a sense of what we're working with.

We'll start with a quick tabulat overview:
- look at the first few rows to see the raw structure,
- check datatypes and missing values,
- review summary statistics like mean, std, and ranges

After that, we'll plot the closing price and trading value over time to spot trends, volatility clusters, and any unusual behaviour.

In [ ]:
# Display the first 5 rows
print("Head of DataFrame:")
print(df.head())

# Concise summary: dtypes, non-null counts, memory usage
print("\n DataFrame Info:")
df.info()

# Descriptive statistics: mean, std, min, max, percentiles
print("\n DataFrame Descriptive Statistics:")
print(df.describe())

In [ ]:
# Plot the close price over time
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['close'], label='Close Price', color='navy')
plt.title('SPY Close Price Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Plot the volume over time
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['volume'], label='Volume', color='darkorange')
plt.title('SPY Trading Volume Over Time')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Shares Traded', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Feature Engineering

Now let's build the targets and the feature matrix using our module functions.

**Why returns (not prices)?**
Returns are closer to stationary and live on a comparable scale across time.
We'll predict next day log returns and also keep a volatility-normalised target for diagnostics.

**What we'll do here**
1) Create targets ('y_raw', 'y_vol') from close prices.
2) Build a lag-safe feature matrix from our raw data (including price, technical, volume, volatility, and statistical blocks) via 'build_feature_matrix'.
3) Run validation checks to guard against common issues (index, NaNs/inf, and obvious lookahead leakage) 
4) Quick preview + visuals

All transforms use past-only information and apply a central lag so that features at timet $t$ predict $t-1$.

In [ ]:
# 1) Targets from close prices
targets = make_targets(df["close"], vol_span=20) # -> columns: log_return, ewma_vol_20, y_raw, y_vol

# 2) Feature matrix (lag-safe; applies a central lag so X_t predicts y_{t+1})
feature_matrix = build_feature_matrix(df, lag_days=1, drop_na_rows=True)

# 3) Validations (index, NaNs/inf, basic anti-leakage structure)
run_all_validations(
    raw=df,
    X=feature_matrix,
    y=targets["y_raw"].reindex(feature_matrix.index), # align to X
    required_raw_cols=("open","high","low","close"),
    expected_central_lag_days=1
)

# 4) Keep a working frame for quick inspection
features = feature_matrix.join(targets[["y_raw", "y_vol"]], how="inner")
print(features.head(10))
print(f"\nFeature shape: {feature_matrix.shape}, with {feature_matrix.columns.nunique()} columns")

### 5.1 What's inside the matrix?

- **Price-based:** spreads (high-low, close-ipen), ratios (close/open), simple & log returns, short-horizon momentum.
- **Technical:** SMA, EMA, RSI(14), MACD(line + signal + histogram), Bollinger Bands (lower, middle, upper).
-  **Volatility & Rolling Stats:** rolling $\sigma$ of returns, rolling mean/std, z-scores.
- **Volume-based:** $\Delta$ volume, volume/avg, OBV, VWAP deviations, trade count stats.
- **Statistical:** Shannon entropy (windows), Hurst exponent (long memory), rolling autocorrelations.

All columns are shifted to $t-1$ within the orchestrator so there's no lookahead.

In [ ]:
print("feature matrix info:")
feature_matrix.info()

print("\nNaN counts (should be 0 after drop_na_rows=true):")
na_counts = feature_matrix.isna().sum().sort_values(ascending=False).head(10)
print(na_counts.to_string())

print("\nTargets preview:")
aligned_targets = targets[["y_raw", "y_vol"]].reindex(feature_matrix.index)
print(aligned_targets.describe().to_string())

### 5.2 Quick Visuals

A couple of lightweight checks help spot obvious issues before modelling:
- **Indicator subplots** on a recent slice (e.g., last 500 days)
- **Correlation heatmap** (high-level view to spot multicollinearity clusters).

In [ ]:
# Indicator subplots

# using last 500 days (to keep plot readable)
df_plot = features.tail(500).copy()

fig, (ax1,  ax2, ax3) = plt.subplots(3, 1, figsize=(15,12), sharex=True)

# Price
ax1.plot(df_plot.index, df.loc[df_plot.index, 'close'], label='Close Price', color='navy')
ax1.set_title("Price with MACD & RSI (last 500 days)")
ax1.set_ylabel("Price ($)")
ax1.legend()
ax1.grid(True, alpha=0.3)

# MACD
ax2.plot(df_plot.index, feature_matrix.loc[df_plot.index, "macd"], label="MACD", color='green')
ax2.set_ylabel("MACD")
ax2.legend()
ax2.grid(True, alpha=0.3)

# RSI
ax3.plot(df_plot.index, feature_matrix.loc[df_plot.index, "rsi_14"], label="RSI (14)", color='darkgreen')
ax3.axhline(70, color='red', linestyle='--', alpha=0.5, label='Overbought (70)')
ax3.axhline(30, color='red', linestyle='--', alpha=0.5, label='Oversold (30)')
ax3.set_ylabel("RSI")
ax3.set_xlabel("Date")
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
FAMILIES = [
    "sma_", "ema_", "rsi_", "macd", "bb_",  # technicals
    "momentum", "simple_return", "log_return",            # price-derived
    "vol_", "rolling_std_", "rolling_mean_", "z_score_",  # volatility/stats
    "obv", "volume_", "volume_over_avg", "trade_count",   # volume
    "entropy", "hurst", "autocorr"                        # statistical
]
cols_to_show = [c for c in feature_matrix.columns if any(prefix in c for prefix in FAMILIES)][:25]

correlation = feature_matrix[cols_to_show].corr().to_numpy()
fig = plt.figure(figsize=(10, 8))
im = plt.imshow(correlation, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Feature Correlation Heatmap (subset)")
plt.xticks(range(len(cols_to_show)), cols_to_show, rotation=90)
plt.yticks(range(len(cols_to_show)), cols_to_show)
plt.tight_layout()
plt.show()  

## 6. Systematic Feature Selection
We've engineered ~40+ features; some help, some don't. Feeding all of them into our LSTM can mask the real patterns, slow training , and hurt generalisation. Instead, we'll rank features on the training window only and freeze a compact set for modelling.

**What we'll do**
1) Define train/val/test** splits (chronological).
2) Fit a Random Forest Regressor on train only.
3) Compute permutation importance on the validation window (model-agnostic reduces Random Forest bias).
4) Pick the top-N features and optionally deduplicate highly correlated ones.
5) Freeze the selected feature list for downstream models.

In [ ]:
# y from targets, aligned to feature_matrix index
y = targets["y_raw"].reindex(feature_matrix.index)

# Keep numeric features only; clean infs; align & drop NaNs
X = (
    feature_matrix
    .select_dtypes(include=[np.number])
    .replace([np.inf, -np.inf], np.nan)
)
X, y = align_X_y(X, y, dropna=True)

# Chronological split: Train / Val / Test (70, 15, 15)
n = len(X)
i_train = int(n * 0.70)
i_val = int(n * 0.85)

X_train, y_train = X.iloc[:i_train], y.iloc[:i_train]
X_val, y_val = X.iloc[i_train:i_val], y.iloc[i_train:i_val]
X_test, y_test = X.iloc[i_val:], y.iloc[i_val:]

print(f"Shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Fit RF on Train ONLY
rf = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    n_jobs=-1,
    random_state=42
).fit(X_train, y_train)

# Permutation importance on VALIDATION, scored by Pearson correlation
r_scorer = make_scorer(pearson_corr, greater_is_better=True)

perm = permutation_importance(
    rf, X_val, y_val,
    n_repeats=20,
    random_state=42,
    scoring=r_scorer
)

imp_df = (
    pd.DataFrame({
        "feature": X_val.columns,
        "importance_mean": perm.importances_mean,
        "importance_std": perm.importances_std
    })
    .sort_values("importance_mean", ascending=False)
    .reset_index(drop=True)
)

# Select top-N and prune near-duplicates
N = 20
top_n_features = imp_df["feature"].tolist()

def prune_redundant(features, Xref, corr_threshold=0.95):
    kept = []
    for feature in features:
        if not kept:
            kept.append(feature); continue
        if all (abs(Xref[feature].corr(Xref[k])) < corr_threshold for k in kept):
            kept.append(feature)
    return kept
    
SELECTED_FEATURES = prune_redundant(top_n_features, X_train, corr_threshold=0.95)
print(f"\nTop {N} by permutation importance:\n {top_n_features}")
print(f"\nAfter redundancy pruning ({len(SELECTED_FEATURES)} kept):\n{SELECTED_FEATURES}")

# Freeze to join JSON for reproducability
ARTIFACTS_DIR = Path("artifacts"); ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
FEATURE_SET_PATH = ARTIFACTS_DIR / "feature_set.json"

payload = {
    "selected_features": SELECTED_FEATURES,
    "meta": {
        "model": "RandomForestRegressor",
        "n_estimators": 600,
        "scoring": "pearson_r",
        "n_repeats": 20,
        "prune_threshold": 0.95,
        "train_range": [str(X_train.index.min()), str(X_train.index.max())],
        "val_range":   [str(X_val.index.min()),   str(X_val.index.max())],
        "random_state": 42,
    },
}
FEATURE_SET_PATH.write_text(json.dumps(payload, indent=2))
print(f"\nSaved frozen feature set → {FEATURE_SET_PATH}")

## §7. Baselines

Before we build the LSTM, let's set simple, transparent baselines to sanity check signal quality and avoid overfitting. These serve as benchmarks that any more sophisticated model should surpass.

What we evaluate
- **Persistence ($r_t -> r_{t+1}$)**: predicts the next day's return from the previous day's return.
- **Ridge Regression (tabular)**: linear model on the frozen feature set from §6. The regularisation parameter $\alpha$ is chosen using validation Pearson correlation (RMSE used as a tiebreaker).
- **XGBoost (tabular)**: tree-based model with early stopping on validation RMSE; we report the best number of trees.

Models are trained on the training split and tuned on the validation split, using Pearson correlation as the primary selection metric and RMSE as a secondary tiebreaker. Validation and test results are reported seperately, with no peeking into the test set.

> *Why Correlation?* Pearson correlation is scale-free and reflects directional/relative accuracy for daily returns. 

In [ ]:
# Compute baseline predictions

# Persistence uses the next-day-aligned target series; at time t ir predicts r_t for r_{t+1}
persistence_pred_val_series = predict_persistence(y_val)
persistence_pred_test_series = predict_persistence(y_test)

# Ridge: fit on Train, select alpha on validation correlation; refit on Train+Val for test
ridge_model, ridge_diagnostics, ridge_val_pred_series = train_ridge_with_val(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    alphas=tuple(np.logspace(-4, 4, 25)),
    refit_on_train_val=True
)
ridge_test_pred_series = predict_series(ridge_model, X_test, name="ridge_pred_test")

# XGBoost: early stopping on validation; refit on train+val (best_n) for test
xgb_model, xgb_diagnostics, xgb_val_pred_series = train_xgb_with_val(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    min_child_weight=1.0,
    early_stopping_rounds=50,
    n_jobs=-1,
    random_state=42,
    refit_on_train_val=True,
)
xgb_test_pred_series = predict_series(xgb_model, X_test, name="xgb_pred_test")

#TODO: MOVE ALL FUNCTIONS TO MODULESA
# Scoring helpers
def score_prediction(true_returns_series: pd.Series, predicted_returns_series:pd.Series):
    """    Score predictions using the project's alignment/cleaning: returns (pearson_r, rmse, directional_accuracy_percent)."""
    # Wrap predictions as a single-column DataFrame
    predicted_df = pd.DataFrame({"predicted": pd.Series(predicted_returns_series)})

    # Align and drop NaNs (post-lag edges, infs handled upstream)
    X_aligned, y_aligned = align_X_y(predicted_df, pd.Series(true_returns_series), dropna=True)

    predicted_aligned_series = X_aligned["predicted"].astype("float64")
    y_aligned_series = y_aligned.astype("float64")

    correlation = pearson_corr(y_aligned_series, predicted_aligned_series)
    rmse_value = float(np.sqrt(mean_squared_error(y_aligned_series, predicted_aligned_series)))
    directional_accuracy = float((np.sign(y_aligned_series) == np.sign(predicted_aligned_series)).mean() * 100.0)

    return correlation, rmse_value, directional_accuracy

# Assemble metrics table
metrics_rows = [
    ["Persistence", 
     *score_prediction(y_val, persistence_pred_val_series),
     *score_prediction(y_test, persistence_pred_test_series)
    ],
    [f"Ridge (alpha={ridge_diagnostics['best_alpha']:.3g})", 
     *score_prediction(y_val, ridge_val_pred_series),
     *score_prediction(y_test, ridge_test_pred_series)
    ],
    [f"XGBoost (n={xgb_diagnostics['best_n_estimators']})", 
     *score_prediction(y_val, xgb_val_pred_series),
     *score_prediction(y_test, xgb_test_pred_series)
    ]
]

baseline_metrics_df = pd.DataFrame(
    metrics_rows,
    columns=[
        "Model",
        "Val Pearson corr", "Val RMSE", "Val Directional Accuracy %",
        "Test Pearson corr", "Test RMSE", "Test Directional Accuracy %"
    ]
).set_index("Model")

formatters = {
    "Val Pearson corr": "{:.3f}".format,
    "Test Pearson corr": "{:.3f}".format,
    "Val RMSE": "{:.3f}".format,
    "Test RMSE": "{:.3f}".format,
    "Val Directional Accuracy %": "{:.2f}".format,
    "Test Directional Accuracy %": "{:.2f}".format,
}
print("§7 Baseline Metrics")
print(baseline_metrics_df.to_string(formatters=formatters))

# inspect chosen hyperparameters at a glance
ridge_diagnostics, xgb_diagnostics

In [ ]:
# 1) Targets from close prices
targets = make_targets(df["close"], vol_span=20) # -> columns: log_return, ewma_vol_20, y_raw, y_vol

# 2) Feature matrix (lag-safe; applies a central lag so X_t predicts y_{t+1})
feature_matrix = build_feature_matrix(df, lag_days=1, drop_na_rows=True)

# 3) Validations (index, NaNs/inf, basic anti-leakage structure)
run_all_validations(
    raw=df,
    X=feature_matrix,
    y=targets["y_raw"].reindex(feature_matrix.index), # align to X
    required_raw_cols=("open","high","low","close"),
    expected_central_lag_days=1
)

# 4) Keep a working frame for quick inspection
features = feature_matrix.join(targets[["y_raw", "y_vol"]], how="inner")
print(features.head(10))
print(f"\nFeature shape: {feature_matrix.shape}, with {feature_matrix.columns.nunique()} columns")

## 8. LSTM Data Preperation & Architecture

We need to turn the frozen feature set into sequences an LSTM can learn from. We'll keep things simple: use the selected colummns, split by time, standardise from the train slice only, window into '(samples, timesteps, features'), and then define a compact LSTM.

In [ ]:
# Load frozen feature list (fallback to all features if not found)
FEATURE_SET_PATH = Path("artifacts/feature_set.json")
if FEATURE_SET_PATH.exists():
    SELECTED_FEATURES = json.loads(FEATURE_SET_PATH.read_text())["selected_features"]
else:
    SELECTED_FEATURES = feature_matrix.columns.tolist()
    print("Frozen feature set not found; using all features.")

# Keep selected numeric features
cols = [col for col in SELECTED_FEATURES if col in feature_matrix.columns]
X_full = feature_matrix[cols].select_dtypes(include=[np.number]).copy()

# Target: next-day log return aligned to X
y_full = targets["y_raw"].reindex(X_full.index).astype("float64")

# Align and drop NaNs (post-lag edges, infs handled upstream)
X_full, y_full = align_X_y(X_full, y_full, dropna=True)

# Subset the dataframes to only include the selected features
X_train_lstm = X_train[SELECTED_FEATURES]
X_val_lstm = X_val[SELECTED_FEATURES]
X_test_lstm = X_test[SELECTED_FEATURES]

# The target series (y_train, y_val, y_test) are already defined and aligned from §6
print(f"\nUsing {len(SELECTED_FEATURES)} features for the LSTM.")

In [ ]:
# Apply Standard Scaler to data

x_scaler = StandardScaler()
y_scaler = StandardScaler()

# Fit scalers on the training data only
x_scaler.fit(X_train_lstm)
y_scaler.fit(y_train.to_frame())

# Transform all splits
X_train_scaler = x_scaler.transform(X_train_lstm)
X_val_scaler = x_scaler.transform(X_val_lstm)
X_test_scaler = x_scaler.transform(X_test_lstm)

y_train_scaler = y_scaler.transform(y_train.to_frame())
y_val_scaler = y_scaler.transform(y_val.to_frame())
y_test_scaler = y_scaler.transform(y_test.to_frame())


In [ ]:
sequence_length = 60 

def make_sequence(X2d: np.ndarray, y2d: np.ndarray, sequence_length: int):
    """Transforms time series data into overlapping sequences for LSTM input."""
    n = len(X2d)
    if n <= sequence_length:
        raise ValueError(f"Not enough rows ({n}) for sequence length {sequence_length}.")
    Xs = np.stack([X2d[i:i+sequence_length, :] for i in range(n - sequence_length)])
    ys = np.stack([y2d[i+sequence_length-1, :] for i in range(n - sequence_length)])
    return Xs.astype("float32"), ys.astype("float32")

# Create the final sequenced arrays for the LSTM model
X_train_seq, y_train_seq = make_sequence(X_train_scaler, y_train_scaler, sequence_length)
X_val_seq, y_val_seq = make_sequence(X_val_scaler, y_val_scaler, sequence_length)
X_test_seq, y_test_seq = make_sequence(X_test_scaler, y_test_scaler, sequence_length)

print("\n--- Final Data Shapes for LSTM ---")
print(f"Shape of training data: X={X_train_seq.shape}, y={y_train_seq.shape}")
print(f"Shape of validation data: X={X_val_seq.shape}, y={y_val_seq.shape}")
print(f"Shape of test data: X={X_test_seq.shape}, y={y_test_seq.shape}")


In [ ]:
# Model architecture
def build_lstm_model(input_shape, learning_rate=1e-3, dropout=0.20):
    model = keras.models.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.LSTM(96, return_sequences=True),
        keras.layers.Dropout(dropout),
        keras.layers.LSTM(48, return_sequences=False),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError(name="rmse")]    
    )
    return model

model = build_lstm_model(input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), learning_rate=1e-3, dropout=0.20)

model.summary()

That gives us windowed, scaled sequences and an LSTM tuned by validation correlation.

## 9.Training & Validation

We optimise with Adam (using a small learning rate for stability) and train for a generous epoch budget while earlystopping on validation Pearson correlation. Pearson is chosen over MSE as it is scale-invariant and better aligned with ranking/forecast objectives.

During training we:
1. compute val_pearson_corr at the end of each epoch
2. early-stop when it plateaus, restoring the best weights, and
3. record the best epoch (the one with highest validation correlation)

After that model selection step, we refit from scratch on Train+Val for exactly best_epoch + 1 epochs, so the final model uses all in-sample data before evaluating on Test. We keep shuffle=False throughout to preserve temporal ordering.

> Notes: 
> - Pearson is "higher is better", so we set mode='max'.
> - We log val_pearson_corr into Keras History.history so it shows in curves.
> - Artefacts and final model weights are archived for reproducability.

In [ ]:
def pearson_corr_pandas(y_true, y_pred) -> float:
    y_true_series = y_true if isinstance(y_true, pd.Series) else pd.Series(np.ravel(y_true))
    y_pred_series = pd.Series(np.ravel(y_pred), index=y_true_series.index)
    corr = pearson_corr(y_true_series, y_pred_series, raise_on_degenerate=False)
    return float(corr) if np.isfinite(corr) else 0.0

# 1. Define the custom metric using TensorFlow/TensorFlow Probability
def pearson_correlation_metric(y_true, y_pred):
    """Pearson correlation metric for TensorFlow tensors."""
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    corr = tfp.stats.correlation(y_true, y_pred, sample_axis=0, event_axis=None)
    # Return 0.0 if correlation is NaN (e.g., due to zero variance)
    return tf.where(tf.math.is_nan(corr), 0.0, corr)

# 2. Update the build_lstm_model function to compile the new metric
def build_lstm_model(input_shape, learning_rate=1e-3, dropout=0.20):
    model = keras.models.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.LSTM(96, return_sequences=True),
        keras.layers.Dropout(dropout),
        keras.layers.LSTM(48, return_sequences=False),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError(name="rmse"), pearson_correlation_metric]    
    )
    return model

# Re-build the model so it includes the new compiled metric
model = build_lstm_model(input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), learning_rate=1e-3, dropout=0.20)

# Run directory
run_stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
ARTEFACTS_DIR = f"artefacts/run-{run_stamp}/"
os.makedirs(ARTEFACTS_DIR, exist_ok=True)

# 3. Update the standard callbacks to monitor the new metric
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_pearson_correlation_metric",
    mode="max",
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_on_plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_pearson_correlation_metric",
    mode="max",
    factor=0.5,
    patience=5,
    verbose=1
)

# 4. Fit the model using the standard callbacks and sequenced data
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=300,
    batch_size=64,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, reduce_on_plateau],
    verbose=1,
    shuffle=False
)

# Get the best epoch and correlation from the training history
best_epoch = np.argmax(history.history.get("val_pearson_correlation_metric", [0]))
best_val_corr = history.history["val_pearson_correlation_metric"][best_epoch]
print(f"\\n[Selection] Best epoch: {best_epoch} | Best val Pearson: {best_val_corr:.6f}")

# Save training history + selection
pd.DataFrame(history.history).to_parquet(f"{ARTEFACTS_DIR}training_history.parquet")
with open(f"{ARTEFACTS_DIR}selection.json", "w") as f:
    json.dump({"best_epoch": int(best_epoch), "best_val_pearson": best_val_corr}, f, indent=2)

# 5. Refit final model on combined SEQUENCED Train+Val data
X_train_val_seq = np.concatenate([X_train_seq, X_val_seq], axis=0)
y_train_val_seq = np.concatenate([y_train_seq, y_val_seq], axis=0)

final_model = build_lstm_model(input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), learning_rate=1e-3, dropout=0.20)

final_history = final_model.fit(
    X_train_val_seq, y_train_val_seq,
    epochs=(best_epoch + 1) if best_epoch is not None else 1,
    batch_size=64,
    verbose=1,
    shuffle=False
)

print(f"\\n[Final Fit] Trained on Train+Val for {(best_epoch + 1) if best_epoch is not None else 1} epoch(s). Ready for test scoring.")

pd.DataFrame(final_history.history).to_parquet(f"{ARTEFACTS_DIR}final_fit_history.parquet")
final_model.save(f"{ARTEFACTS_DIR}final_model.keras")

# 6. Make Test Predictions using SEQUENCED test data
y_test_pred_scaled = final_model.predict(X_test_seq, verbose=0).reshape(-1)
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()
test_corr = pearson_corr_pandas(y_test_seq.flatten(), y_test_pred)
print(f"\\n[Test] Pearson correlation: {test_corr:.6f}")

pd.DataFrame({"y_true": np.ravel(y_test_seq), "y_pred": y_test_pred}).to_parquet(f"{ARTEFACTS_DIR}test_predictions.parquet")
with open(f"{ARTEFACTS_DIR}test_results.json", "w") as f:
    json.dump({"pearson_corr": test_corr}, f, indent=2)

# Make Validation predictions for diagnostics (from initial early-stopped model)
val_pred_scaled = model.predict(X_val_seq, verbose=0).reshape(-1)
val_pred_best = y_scaler.inverse_transform(val_pred_scaled.reshape(-1, 1)).flatten()
pd.DataFrame({"y_true": np.ravel(y_val_seq), "y_pred": val_pred_best}).to_parquet(f"{ARTEFACTS_DIR}val_results.parquet")


## 10. Test Evaluation & Backtest

Once the model is trained and predictions are available for the test set, we map predicted signals to positions and evaluate their performance relative to several baselines and a label-shuffle control. The evaluation quantifies both return and risk characteristics of the strategies.

For each model and control:
1. **Backtest**: convert predictions into positions and compute net returns using a single-asset backtest framework with realistic trading parameters, including transaction costs, turnover limits, and optional volatility targeting.
2. **Wealth Curve**: generate a $1,000 wealth curve for the strategy and compare it to a Buy 7 Hold benchmark.
3. **Metrics**: compute key performance statistics including Sharpe ratio, maximum drawdown (MaxDD), turnover, and other relevant summaries. Additional analyses include trade-level summaries, long/short attribution, yearly returns, and regime-based performance by realised volatility.
4. **Controls**: Include baselines such as persistence, Ridge, XGBoost predictions, as well as a label-shuffle control for sanity check.

All results are aggregated into concise tables and wealth charts, and artefacts (summary statistics, wealth series, and auxiliary tables) are persisted for reproducability. This setup provides a rigorous framework to assess predictive signal quality, trading robustness, and economic significance of model outputs on out-of-sample data.

In [ ]:
# Get the correct index for the test data after sequencing.
# The index starts after the first 'sequence_length - 1' observations are used for the first sequence.
test_idx = X_test.index[sequence_length - 1:]

# Inverse transform the predictions and true values to get their original log-return scale.
y_test_pred_unscaled = y_scaler.inverse_transform(y_test_pred.reshape(-1, 1)).flatten()
y_test_true_unscaled = y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Create correctly indexed pandas Series.
pred_lstm_test = pd.Series(y_test_pred_unscaled, index=test_idx, name="lstm_pred_test")
y_test_series_log = pd.Series(y_test_true_unscaled, index=test_idx, name="y_test_log")

# Align all baseline predictions to the same index as the LSTM for a fair comparison.
persistence_pred_test_series = persistence_pred_test_series.reindex(test_idx)
ridge_test_pred_series = ridge_test_pred_series.reindex(test_idx)
xgb_test_pred_series = xgb_test_pred_series.reindex(test_idx)

# Helper to run a backtest and summarise
def backtest_and_summarize(scores: pd.Series, y_next_log: pd.Series, name: str):
    """Runs backtest, generates wealth curve, and summarises performance."""
    
    # Convert log returns to simple returns for accurate PnL compounding in the backtest.
    y_next_simple = np.exp(y_next_log) - 1
    
    backtest = single_asset_backtest(
        scores=scores,
        y_raw_next=y_next_simple, # Pass simple returns here
        z_window=60,
        raw_weight_cap=1.0,
        target_vol_annual=0.10,
        vol_span=20,
        cost_bps=5.0,
        gross_limit=3.0,
        execution="close_to_close",
        drop_warmup=True,
        use_vol_targeting=True,
    )
    
    # Align backtest results with target series to ensure consistent length
    X_aligned, y_aligned_log = align_X_y(backtest[["net_return"]], y_next_log, dropna=False)
    backtest_aligned = backtest.loc[X_aligned.index]
    
    # Wealth (strategy) is calculated from the simple net returns of the backtest
    wealth_strat = wealth_curve(
        returns=backtest_aligned["net_return"],
        starting_wealth=1_000.0,
        returns_are_log=False
    ).rename(name)
    
    # Backtest summary
    summary = summarize_backtest(
        backtest_frame=backtest_aligned,
        return_column="net_return",
        turnover_column="turnover",
        starting_wealth=1_000.0,
        returns_are_log=False,
        label=name,
        fold="test"
    )
    
    # Note: The wealth_bh (Buy&Hold) is now calculated outside this helper for robustness.
    return {"name": name, "wealth_strat": wealth_strat, "summary": summary}

# Run for LSTM + baselines + label-shuffle control 
results = []
results.append(backtest_and_summarize(pred_lstm_test, y_test_series_log, "LSTM"))
results.append(backtest_and_summarize(persistence_pred_test_series, y_test_series_log, "Persistence"))
results.append(backtest_and_summarize(ridge_test_pred_series, y_test_series_log, "Ridge"))
results.append(backtest_and_summarize(xgb_test_pred_series, y_test_series_log, "XGBoost"))

# Label-shuffle control (sanity check)
rng = np.random.default_rng(42)
scores_shuffle = pd.Series(rng.permutation(pred_lstm_test.values), index=pred_lstm_test.index, name="Label-Shuffle")
results.append(backtest_and_summarize(scores_shuffle, y_test_series_log, "Label-Shuffle"))

# Aggregate wealth curves for the plot/table 
wealth_df = pd.concat([result["wealth_strat"] for result in results], axis=1)

# Robustly calculate the Buy & Hold wealth curve from the original `targets` DataFrame
buy_hold_returns = targets['y_raw'].reindex(wealth_df.index).dropna()
wealth_bh = wealth_curve(
    returns=buy_hold_returns,
    starting_wealth=1_000.0,
    returns_are_log=True  # We know 'y_raw' contains log returns
).rename("Buy&Hold")

# Combine all wealth curves for plotting
wealth_df = pd.concat([wealth_df, wealth_bh], axis=1)

# Build concise stats table
summary_table = pd.concat([result["summary"].rename(result["name"]) for result in results]).T

# Compute Buy & Hold metrics
bh_return_series = targets['y_raw'].reindex(wealth_df.index).dropna()
bh_row = pd.Series({
    "cagr": float(annualized_return(bh_return_series, returns_are_log=True)),
    "vol": float(annualized_volatility(bh_return_series)),
    "sharpe": float(sharpe_ratio(bh_return_series, risk_free_rate_annual=0.0)),
    "max_drawdown": float(max_drawdown(wealth_bh)[0]),
    "hit_rate": float(hit_rate(bh_return_series)),
    "turnover_mean": np.nan,
    "label": "Buy&Hold",
    "fold": "test"
}, name="Buy&Hold")

# Join columns then append Buy & Hold
stats_df = pd.concat([summary_table, bh_row.to_frame().T], axis=0)

# Display the stats table
print("Backtest & Test Evaluation — key stats (test window)")
table = stats_df.rename(columns={
    "cagr": "CAGR", "vol": "Vol", "sharpe": "Sharpe",
    "max_drawdown": "MaxDD", "turnover_mean": "Turnover (mean)", "hit_rate": "HitRate"
    }).style.format({
    "CAGR": "{:.2%}", "Vol": "{:.2%}", "Sharpe": "{:.2f}",
    "MaxDD": "{:.2%}", "Turnover (mean)": "{:.4f}", "HitRate": "{:.2%}"
}).set_caption("Backtest Summary")

table

# $1k Wealth Chart
plt.figure(figsize=(10, 5))
wealth_df.plot(ax=plt.gca())
plt.title("$1k Wealth — Test Window")
plt.xlabel("Date")
plt.ylabel("Wealth ($)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Conclusions & Next Steps

## 10. Finale: Findings, Limitations & Next Steps

### 10.1 Summary of What We Built
This project assembled a **time‑series forecasting pipeline** for SPY daily data:
- Retrieved and cleaned OHLCV via Alpaca; engineered **price/ratio/volatility**, **technical indicators** (SMA/EMA, RSI, MACD, Bollinger), plus **statistical** features (Shannon entropy, Hurst exponent, ACF).
- Performed **time‑aware feature selection** with a Random Forest to identify the most informative signals for next‑day returns.
- Prepared **chronological train/val/test splits**, fit scalers on **train only** to avoid leakage, and created **fixed‑length sequences** for the LSTM.
- Trained a **2-layer LSTM** with dropout and early stopping; evaluated both **return prediction metrics** and **price diagnostics** (anchored and chained paths), plus **directional accuracy**, **residuals**, and **stability**.

### 10.2 Key Takeaways (Interpretation)
- **Return predictions** show a very small R² (~0.0075) — expected in a near-efficient market — but a **directional accuracy of ~60%** suggests the model captures some signal in predicting the sign of returns, even if magnitude predictions are noisy.  
- **Anchored price path** reconstruction yields a high R² (~0.957) and low MAPE (~0.52%), but this is partly an artefact of using the last known actual price each step — it’s optimistic and less representative of real deployment.  
- **Chained price path**, which compounds predicted returns without resetting to actuals, shows more realistic performance: R² drops to ~0.644 and errors are larger (MAE ~10.29, RMSE ~12.35). This better reflects the model’s live trading performance, where prediction errors accumulate.  
- **Magnitude vs. direction**: While predicting exact next-day returns remains challenging (high MAPE in returns), the directional edge could be exploitable in a strategy with good risk management.  

Overall, the model appears better suited for **trend direction signalling** rather than precise return magnitude forecasting.

### 10.3 Limitations & Risks
- **Non‑stationarity / regime shifts**: Relationships change across time (macro cycles, volatility regimes). A static model can decay quickly without **walk‑forward re‑training**.
- **Label definition**: Next‑day return may be too noisy; alternative horizons (e.g., 5‑day) or targets (e.g., **directional** labels) could be more learnable.
- **Evaluation bias**: Anchored price diagnostics can look optimistic; **chained** paths are more realistic and should be emphasised.
- **Data coverage**: Using only SPY daily bars leaves **news/macro/term structure** unexplored; edge likely improves with richer context.

### 10.4 Next Steps (High‑Impact First)
1. **Walk‑Forward Validation**  
   Switch to rolling-window training & evaluation (expand/slide) to mimic live deployment. Log metrics over each window.
2. **Hyperparameter Search**  
   Tune sequence length, hidden sizes, dropout, learning rate, and batch size (e.g., KerasTuner or Optuna). Start LR at 1e‑3/1e‑4.
3. **Targets & Horizons**  
   Try directional targets (`sign(return)`) and multi‑horizon returns (e.g., 1/3/5 day). Compare hit rate, precision/recall, and calibration.
4. **Features**  
   Add **trend/vol regime flags**, **market breadth**, **term structure** (VIX term, yield curve), **seasonality dummies**, and **macro surprise** proxies. Re‑run selection.
5. **Baselines & Ensembling**  
   Add simple baselines (zero‑return, yesterday’s sign, SMA crossover), linear/elastic‑net, XGBoost/LightGBM, and a TCN. Consider simple **averaging or stacking**.
6. **Robustness & Stability**  
   Bootstrap test periods, compute **confidence intervals** for metrics, and add **drift detection** (e.g., PSI) to know when to retrain.
7. **Reproducibility & Ops**  
   Seed everything (NumPy/TF/Python), log versions, and save the **scalers + model** artifacts. Add a small **inference notebook**.
